## 讀檔

In [1]:
import json

num = 0
train_data = []
test_data = []
train_size = 5000
test_size = 100

input_file = open('translation2019zh_train.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    if num > train_size + test_size:
        break
    elif num > train_size:
        test_data.append(json.loads(line))
    else:
        train_data.append(json.loads(line))
input_file.close()

## 簡繁體轉換

In [3]:
from opencc import OpenCC

cc = OpenCC('s2t')

for data in train_data:
    data['chinese'] = cc.convert(data['chinese'])

## 整理為中文及英文字典

In [5]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

## 字典排序

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

## 計算編碼、解碼器的最大長度

In [7]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

## 儲存字典單字及序號

In [11]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print(input_token_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, '\\': 59, ']': 60, '^': 61, '_': 62, '`': 63, 'a': 64, 'b': 65, 'c': 66, 'd': 67, 'e': 68, 'f': 69, 'g': 70, 'h': 71, 'i': 72, 'j': 73, 'k': 74, 'l': 75, 'm': 76, 'n': 77, 'o': 78, 'p': 79, 'q': 80, 'r': 81, 's': 82, 't': 83, 'u': 84, 'v': 85, 'w': 86, 'x': 87, 'y': 88, 'z': 89, '~': 90, '\xad': 91, '´': 92, '·': 93, 'Â': 94, 'á': 95, 'â': 96, 'ä': 97, 'è': 98, 'é': 99, 'ñ': 100, 'ó': 101, 'ö': 102, 'ü': 103, 'δ': 104, '–': 105, '—': 106, '‘': 107, '’': 108, '“': 109, '”': 

In [13]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, 'A': 33, 'B': 34, 'C': 35, 'D': 36, 'E': 37, 'F': 38, 'G': 39, 'H': 40, 'I': 41, 'J': 42, 'K': 43, 'L': 44, 'M': 45, 'N': 46, 'O': 47, 'P': 48, 'Q': 49, 'R': 50, 'S': 51, 'T': 52, 'U': 53, 'V': 54, 'W': 55, 'X': 56, 'Y': 57, 'Z': 58, '[': 59, '\\': 60, ']': 61, '^': 62, '_': 63, 'a': 64, 'b': 65, 'c': 66, 'd': 67, 'e': 68, 'f': 69, 'g': 70, 'h': 71, 'i': 72, 'j': 73, 'k': 74, 'l': 75, 'm': 76, 'n': 77, 'o': 78, 'p': 79, 'q': 80, 'r': 81, 's': 82, 't': 83, 'u': 84, 'v': 85, 'w': 86, 'x': 87, 'y': 88, 'z': 89, '~': 90, '\xad': 91, '·': 92, 'Ñ': 93, 'á': 94, 'ñ': 95, 'ö': 96, 'ü': 97, '–': 98, '—': 99, '‘': 100, '’': 101, '“': 102, '”': 103, '•': 104, '…': 105, '‧': 106, '‰': 107, '℃': 108, '①': 109, '─'

## 設定編碼器、解碼器input起始值

In [10]:
import numpy as np

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

## 設定 encoder_input、decoder_input對應的順序 

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

## 建立 encoder LSTM 隱藏層

In [24]:
from keras.layers import Input, LSTM, Dense
import tensorflow as tf

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

origin_encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_0')
origin_encoder = LSTM(latent_dim, return_state=True)
origin_encoder_outputs, origin_state_h, origin_state_c = origin_encoder(origin_encoder_inputs)
origin_encoder_states = [origin_state_h, origin_state_c]

encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_1')
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_outputs.type_spec

TensorSpec(shape=(None, 256), dtype=tf.float32, name=None)

In [ ]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]